In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-06 01:00:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.94MB/s    in 0.2s    

2021-09-06 01:00:17 (5.94 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [6]:
#Recreating vine_table
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

In [9]:
#All rows where total_votes are >= 20
vine_table_20 = vine_table.filter("total_votes>=20")
vine_table_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R2JK5Y8D5MXAGP|          1|            5|        121|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1H5Y9Z2DHRNVO|          1|            4|         95|   N|                Y|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|     

In [15]:
#Df filtered there ratio of helpful_votes to total_votes is >= 50%
vine_table_ratio = vine_table_20.filter("helpful_votes/total_votes >= 0.5")
vine_table_ratio.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

In [17]:
# filtering to show the rows where the reviews were and were not part of the vine program
vine_table_paid = vine_table_ratio.filter("vine == 'Y'")

vine_table_unpaid = vine_table_ratio.filter("vine == 'N'")

vine_table_paid.show()
vine_table_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BOQTLUU3Y4L8|          4|           42|         55|   Y|                N|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|
| REPU28WG1VZUE|          5|          242|        281|   Y|                N|
|R1D6REC9HPJVQY|          4|           31|         41|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|    

In [58]:
vine_count = vine_table_ratio.groupby("vine").agg({"review_id": "count"})
vine_count.show()

+----+----------------+
|vine|count(review_id)|
+----+----------------+
|   Y|               4|
|   N|            1064|
+----+----------------+



In [55]:
#determining total number of reviews, number of 5-star reviews and percentage of 5-star reviers to the two types of review
number_of_reviews = vine_table_ratio.agg({"review_id": "count"}).first()['count(review_id)']

print(f"Total number of reviews: {number_of_reviews}")

number_of_5_stars = vine_table_ratio.groupby("star_rating").agg({"review_id": "count"}).orderBy(vine_table_ratio["star_rating"].desc()).first()["count(review_id)"]

print(f"Total number of 5-star reviews: {number_of_5_stars}")

vine_to_5star = vine_table_ratio.filter("star_rating=='5'").groupby("vine").agg({"review_id": "count"}).withColumnRenamed("count(review_id)", "Number of 5-star reviews")

vine_to_star = vine_to_5star.withColumn("% of 5 star reviews", vine_to_5star['Number of 5-star reviews']/number_of_5_stars)

vine_to_star.show()

Total number of reviews: 1068
Total number of 5-star reviews: 528
+----+------------------------+--------------------+
|vine|Number of 5-star reviews| % of 5 star reviews|
+----+------------------------+--------------------+
|   Y|                       1|0.001893939393939394|
|   N|                     527|  0.9981060606060606|
+----+------------------------+--------------------+



In [77]:
vine_count_all_review = vine_table_ratio.groupby("star_rating", "vine").agg({"review_id": "count"}).orderBy(vine_table_ratio["star_rating"].desc())

star_count = vine_table_ratio.groupby("star_rating").agg({"review_id": "count"}).orderBy(vine_table_ratio["star_rating"].desc())

vine_count_all_review.show()

+-----------+----+----------------+
|star_rating|vine|count(review_id)|
+-----------+----+----------------+
|          5|   Y|               1|
|          5|   N|             527|
|          4|   Y|               2|
|          4|   N|             182|
|          3|   Y|               1|
|          3|   N|              95|
|          2|   N|              63|
|          1|   N|             197|
+-----------+----+----------------+

